In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# /Users/chriswesterman/Library/Jupyter/nbextensions/snippets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import beatthebookies.data as btb

pd.set_option('display.max_columns', 500)


# Exploratory notebook

In [8]:
test = btb.get_data(season='2008/2009')

get_data 6.78


In [9]:
test.shape

(380, 31)

In [10]:
seasons = ['2009/2010','2010/2011', '2011/2012', '2012/2013', '2013/2014', '2014/2015', '2015/2016']

for year in seasons:
    x = btb.get_data(season=year)
    test = pd.concat([test, x], axis=0)

get_data 6.81
get_data 6.82
get_data 6.8
get_data 6.87
get_data 6.91
get_data 6.92
get_data 6.84


In [11]:
test.shape

(3040, 31)

In [13]:
test.tail()

,id,season,stage,date,home_team,away_team,home_team_goal,away_team_goal,home_w,away_w,draw,home_t_home_goals,home_t_total_goals,home_t_home_goals_against,home_t_total_goals_against,home_t_home_wins,home_t_home_losses,home_t_home_draws,home_t_total_wins,home_t_total_losses,home_t_total_draws,away_t_away_goals,away_t_total_goals,away_t_away_goals_against,away_t_total_goals_against,away_t_away_wins,away_t_away_losses,away_t_away_draws,away_t_total_wins,away_t_total_losses,away_t_total_draws
312,4701,2015/2016,38,2016-05-15 00:00:00,Everton,Norwich City,3,0,1,0,0,32,56,30,55,5,8,5,10,13,14,13,39,34,64,3,13,2,9,21,7
311,4700,2015/2016,38,2016-05-15 00:00:00,Chelsea,Leicester City,1,1,0,0,1,31,58,29,52,5,5,8,12,12,13,32,67,17,35,11,2,5,23,3,11
310,4699,2015/2016,38,2016-05-15 00:00:00,Arsenal,Aston Villa,4,0,1,0,0,27,61,11,36,11,3,4,19,7,11,13,27,37,72,1,14,3,3,26,8
315,4704,2015/2016,38,2016-05-15 00:00:00,Southampton,Crystal Palace,4,1,1,0,0,35,55,21,40,10,5,3,17,11,9,19,38,24,47,5,7,6,11,17,9
313,4702,2015/2016,38,2016-05-17 00:00:00,Manchester United,Bournemouth,3,1,1,0,0,24,46,8,34,11,2,5,18,10,9,21,44,30,64,6,8,4,11,17,9
